In [1]:
# Make division default to floating-point, saving confusion
from __future__ import division
from __future__ import print_function

# ordered dictionaries are useful for keeping ordered sets of variables
from collections import OrderedDict as odict
# combinatorics
from itertools import product
# table formating for screen output
from tabulate import tabulate

import pandas as pd
import numpy as np
import pickle
import math

In [2]:
def printFactor(f):
    """
    argument 
    `f`, a factor to print on screen. If the `ext` field is presented, it will be printed as an additional column
    """
    # Create a empty list that we will fill in with the probability table entries
    table = list()
    
    # Iterate over all keys and probability values in the table
    for key, item in f['table'].items():
        # Convert the tuple to a list to be able to manipulate it
        k = list(key)
        # Append the probability value to the list with key values
        k.append(item)
        
        # NEW CODE
        # if the 'ext' key is present in the factor, we apppend the ext dictionary converted to a list of items
        if 'ext' in f:
            k.append(list(f['ext'][key].items()))
        
        # Append an entire row to the table
        table.append(k)
    # dom is used as table header. We need it converted to list
    dom = list(f['dom'])
    # Append a 'Pr' to indicate the probabity column
    dom.append('Pr')
    
    # NEW CODE
    # Append a 'Ext' to indicate the extended column
    dom.append('Ext')
    print(tabulate(table,headers=dom,tablefmt='orgtbl'))
    
def prob(factor, *entry):
    """
    argument 
    `factor`, a dictionary of domain and probability table,
    `entry`, a list of values, one for each variable, in the same order as specified in the factor domain.
    
    Returns p(entry)
    """
    return factor['table'][entry]

def evidence(var, obs, outcomeSpace):
    """
    argument 
    `var`, a valid variable identifier.
    `e`, the observed value for var.
    `outcomeSpace`, dictionary with the domain of each variable
    
    Returns dictionary with a copy of outcomeSpace with var = e
    """
    newOutcomeSpace = outcomeSpace.copy()             # Make a copy of outcomeSpace with a copy to method copy(). 1 line
    newOutcomeSpace[var] = tuple(obs)        # Replace the domain of variable var with a tuple with a single element e. 1 line
    return newOutcomeSpace

def join(f1, f2, outcomeSpace):
    """
    argument 
    `f1`, first factor to be joined.
    `f2`, second factor to be joined.
    `outcomeSpace`, dictionary with the domain of each variable
    
    Returns a new factor with a join of f1 and f2
    """
    
    # First, we need to determine the domain of the new factor. It will be union of the domain in f1 and f2
    # But it is important to eliminate the repetitions
    common_vars = list(f1['dom']) + list(set(f2['dom']) - set(f1['dom']))
    
    # We will build a table from scratch, starting with an empty list. Later on, we will transform the list into a odict
    table = list()
    
    # Here is where the magic happens. The product iterator will generate all combinations of varible values 
    # as specified in outcomeSpace. Therefore, it will naturally respect observed values
    for entries in product(*[outcomeSpace[node] for node in common_vars]):
        
        # We need to map the entries to the domain of the factors f1 and f2
        entryDict = dict(zip(common_vars, entries))
        f1_entry = (entryDict[var] for var in f1['dom'])
        f2_entry = (entryDict[var] for var in f2['dom'])
        
        p1 = prob(f1, *f1_entry)           # Use the fuction prob to calculate the probability in factor f1 for entry f1_entry 
        p2 = prob(f2, *f2_entry)           # Use the fuction prob to calculate the probability in factor f2 for entry f2_entry 
        
        # Create a new table entry with the multiplication of p1 and p2
        table.append((entries, p1 * p2))
    return {'dom': tuple(common_vars), 'table': odict(table)}

def log_join(f1, f2, outcomeSpace):
    """
    argument 
    `f1`, first factor to be joined.
    `f2`, second factor to be joined.
    `outcomeSpace`, dictionary with the domain of each variable
    
    Returns a new factor with a join of f1 and f2
    """
    
    # First, we need to determine the domain of the new factor. It will be union of the domain in f1 and f2
    # But it is important to eliminate the repetitions
    common_vars = list(f1['dom']) + list(set(f2['dom']) - set(f1['dom']))
    
    # We will build a table from scratch, starting with an empty list. Later on, we will transform the list into a odict
    table = list()
    
    # Here is where the magic happens. The product iterator will generate all combinations of varible values 
    # as specified in outcomeSpace. Therefore, it will naturally respect observed values
    for entries in product(*[outcomeSpace[node] for node in common_vars]):
        
        # We need to map the entries to the domain of the factors f1 and f2
        entryDict = dict(zip(common_vars, entries))
        f1_entry = (entryDict[var] for var in f1['dom'])
        f2_entry = (entryDict[var] for var in f2['dom'])
        
        p1 = prob(f1, *f1_entry)           # Use the fuction prob to calculate the probability in factor f1 for entry f1_entry 
        p2 = prob(f2, *f2_entry)           # Use the fuction prob to calculate the probability in factor f2 for entry f2_entry 
        
        # Create a new table entry with the multiplication of p1 and p2
        if p1 > 0:
            p1=math.log(p1)
        else:
            p1=-math.inf
            
        if p2 > 0:
            p2=math.log(p2)
        else:
            p2=-math.inf
            
        table.append((entries, p1 + p2))
    return {'dom': tuple(common_vars), 'table': odict(table)}



def marginalize(f, var, outcomeSpace):
    """
    argument 
    `f`, factor to be marginalized.
    `var`, variable to be summed out.
    `outcomeSpace`, dictionary with the domain of each variable
    
    Returns a new factor f' with dom(f') = dom(f) - {var}
    """    
    
    # Let's make a copy of f domain and convert it to a list. We need a list to be able to modify its elements
    new_dom = list(f['dom'])
    
    #########################
    # Insert your code here #
    #########################
    new_dom.remove(var)                          # Remove var from the list new_dom by calling the method remove(). 1 line
    table =list()                   # Create an empty list for table. We will fill in table from scratch. 1 line
    for entries in product(*[outcomeSpace[node] for node in new_dom]):
        s = 0;                  # Initialize the summation variable s. 1 line
        
        
        # We need to iterate over all possible outcomes of the variable var
        for val in outcomeSpace[var]:
            # To modify the tuple entries, we will need to convert it to a list
            entriesList = list(entries)
            # We need to insert the value of var in the right position of entriesList
            entriesList.insert(f['dom'].index(var), val)
            

            #########################
            # Insert your code here #
            #########################
            p = prob(f, *entriesList)                             # Calculate the probability of factor f for entriesList. 1 line
            s +=p                        # Sum over all values of var by accumulating the sum in s. 1 line
            
        # Create a new table entry with the multiplication of p1 and p2
        table.append((entries, s))
    return {'dom': tuple(new_dom), 'table': odict(table)}

def normalize(f):
    """
    argument 
    `f`, factor to be normalized.
    
    Returns a new factor f' as a copy of f with entries that sum up to 1
    """ 
    new_f=f.copy()
    total=0
    for elem in new_f['table']:
        total+=new_f['table'][elem]
    for elem in new_f['table']:
        new_f['table'][elem]=f['table'][elem]/total
        
    return new_f

def allEqualThisIndex(dict_of_arrays, **fixed_vars):
    """
    Helper function to create a boolean index vector into a tabular data structure,
    such that we return True only for rows of the table where, e.g.
    column_a=fixed_vars['column_a'] and column_b=fixed_vars['column_b'].
    
    This is a simple task, but it's not *quite* obvious
    for various obscure technical reasons.
    
    It is perhaps best explained by an example.
    
    >>> all_equal_this_index(
    ...    {'X': [1, 1, 0], Y: [1, 0, 1]},
    ...    X=1,
    ...    Y=1
    ... )
    [True, False, False]
    """
    # base index is a boolean vector, everywhere true
    first_array = dict_of_arrays[list(dict_of_arrays.keys())[0]]
    index = np.ones_like(first_array, dtype=np.bool_)
    for var_name, var_val in fixed_vars.items():
        index = index & (np.asarray(dict_of_arrays[var_name])==var_val)
    return index

def estProbTable(data, var_name, parent_names, outcomeSpace):
    """
    Calculate a dictionary probability table by ML given
    `data`, a dictionary or dataframe of observations
    `var_name`, the column of the data to be used for the conditioned variable and
    `parent_names`, a tuple of columns to be used for the parents and
    `outcomeSpace`, a dict that maps variable names to a tuple of possible outcomes
    Return a dictionary containing an estimated conditional probability table.
    """    
    
    var_outcomes = outcomeSpace[var_name]
    parent_outcomes = [outcomeSpace[var] for var in (parent_names)]
    
    # cartesian product to generate a table of all possible outcomes
    all_parent_combinations = product(*parent_outcomes)

    prob_table = odict()
    
    for i, parent_combination in enumerate(all_parent_combinations):
        parent_vars = dict(zip(parent_names, parent_combination))
        parent_index = allEqualThisIndex(data, **parent_vars)
        for var_outcome in var_outcomes:
            var_index = (np.asarray(data[var_name])==var_outcome)
            if parent_index.sum()==0.0:
                prob_table[tuple(list(parent_combination)+[var_outcome])] = 0
            else:
                prob_table[tuple(list(parent_combination)+[var_outcome])] = (var_index & parent_index).sum()/parent_index.sum()   
    return {'dom': tuple(list(parent_names)+[var_name]), 'table': prob_table}

def log_estProbTable(data, var_name, parent_names, outcomeSpace):
    """
    Calculate a dictionary probability table by ML given
    `data`, a dictionary or dataframe of observations
    `var_name`, the column of the data to be used for the conditioned variable and
    `parent_names`, a tuple of columns to be used for the parents and
    `outcomeSpace`, a dict that maps variable names to a tuple of possible outcomes
    Return a dictionary containing an estimated conditional probability table.
    """    
    
    var_outcomes = outcomeSpace[var_name]
    parent_outcomes = [outcomeSpace[var] for var in (parent_names)]
    
    # cartesian product to generate a table of all possible outcomes
    all_parent_combinations = product(*parent_outcomes)

    prob_table = odict()
    
    for i, parent_combination in enumerate(all_parent_combinations):
        parent_vars = dict(zip(parent_names, parent_combination))
        parent_index = allEqualThisIndex(data, **parent_vars)
        for var_outcome in var_outcomes:
            var_index = (np.asarray(data[var_name])==var_outcome)
            if parent_index.sum()==0.0:
                prob_table[tuple(list(parent_combination)+[var_outcome])] = -math.inf
            else:
                x=(var_index & parent_index).sum()/parent_index.sum()
                prob_table[tuple(list(parent_combination)+[var_outcome])] = math.log(x) if x!=0 else -math.inf 
    return {'dom': tuple(list(parent_names)+[var_name]), 'table': prob_table}


def full_joint(prob_tables, outcomeSpace, res):
    if len(res) < 1:
        return {}
    if len(res) ==1:
        return prob_tables[res[0]]
    if len(res) ==2:
        return join(prob_tables[res[0]],prob_tables[res[1]],outcomeSpace)
    length = len(res)
    half = int(len(res)/2)
    left = full_joint(prob_tables,outcomeSpace,res[0:half])
    right = full_joint(prob_tables,outcomeSpace,res[half:length])
    print(right)
    return join(left,right,outcomeSpace)

def transposeGraph(G):
    """
    argument 
    `G`, an adjacency list representation of a graph
    """      
    GT={}
    for v in G:
        for w in G[v]:
            if w in GT:
                GT[w].append(v)
            else:
                GT[w]=[v]
    return GT



def maximize(f, var, outcomeSpace):
    """
    argument 
    `f`, factor to be maximized.
    `var`, variable to be maximized out.
    `outcomeSpace`, dictionary with the domain of each variable.
    
    Returns a new factor f' with dom(f') = dom(f) - {var}. The `ext` field is updated to store the most likely instantiation
    """    
    
    # Let's make a copy of f domain and convert it to a list. We need a list to be able to modify its elements
    new_dom = list(f['dom']) 
    new_dom.remove(var)            # Remove var from the list new_dom by calling the method remove(). 1 line
    new_ext=list()
    table = list()                 # Create an empty list for table. We will fill in table from scratch. 1 line
    for entries in product(*[outcomeSpace[node] for node in new_dom]):
        m = -1;                    # Initialize the maximization variable m. 1 line

        # We need to iterate over all possible outcomes of the variable var
        for val in outcomeSpace[var]:
            # To modify the tuple entries, we will need to convert it to a list
            entriesList = list(entries)
            # We need to insert the value of var in the right position in entriesList
            entriesList.insert(f['dom'].index(var), val)

            p = prob(f, *tuple(entriesList))     # Calculate the probability of factor f for entriesList. 1 line
            if p > m:
                m=p
                max_val=val
                if ('ext' in f):
                    max_ext=f['ext'][tuple(entriesList)].copy()
                else:
                    max_ext=dict()
            m = max(m, p)                        # Maximize over all values of var by storing the max value in m. 1 line
            
        # Create a new table entry with the multiplication of p1 and p2
        table.append((entries, m))
        max_ext[var]=max_val
        new_ext.append((entries, max_ext))
    return {'dom': tuple(new_dom), 'table': odict(table), 'ext': odict(new_ext)}

In [3]:
def t_log_estProbTable(data, var_name, parent_names, outcomeSpace):
    """
    Calculate a dictionary probability table by ML given
    `data`, a dictionary or dataframe of observations
    `var_name`, the column of the data to be used for the conditioned variable and
    `parent_names`, a tuple of columns to be used for the parents and
    `outcomeSpace`, a dict that maps variable names to a tuple of possible outcomes
    Return a dictionary containing an estimated conditional probability table.
    """    
    var_outcomes = outcomeSpace[var_name]

    dic={}
    for parent in list(parent_names):
        
    # cartesian product to generate a table of all possible outcomes
        parent_outcomes = [outcomeSpace[parent]]
        all_parent_combinations = product(*parent_outcomes)
        prob_table = odict()

        for i, parent_combination in enumerate(all_parent_combinations):
            parent_vars = dict(zip([parent], parent_combination))
            parent_index = allEqualThisIndex(data[1:], **parent_vars)
            for var_outcome in var_outcomes:
                var_index = (np.asarray(data[:-1][var_name])==var_outcome)
                if parent_index.sum()==0.0:
                    prob_table[tuple(list(parent_combination)+[var_outcome])] = -math.inf
                else:
                    x=(var_index & parent_index).sum()/parent_index.sum()
                    prob_table[tuple(list(parent_combination)+[var_outcome])] = math.log(x) if x!=0 else -math.inf
        dic[parent]={'dom': tuple([parent]+[var_name+'_t']), 'table': prob_table}
    return dic

def t_log_learn_bayes_net(G, data, outcomeSpace):
    prob_tables={}
    for node in G:
        cond=G[node]
        prob_tables[node]=t_log_estProbTable(data, node, cond, outcomeSpace)
    return prob_tables

In [4]:
def t_sensor_log_estProbTable(data, var_name, parent_names, outcomeSpace):
    """
    Calculate a dictionary probability table by ML given
    `data`, a dictionary or dataframe of observations
    `var_name`, the column of the data to be used for the conditioned variable and
    `parent_names`, a tuple of columns to be used for the parents and
    `outcomeSpace`, a dict that maps variable names to a tuple of possible outcomes
    Return a dictionary containing an estimated conditional probability table.
    """    
    var_outcomes = outcomeSpace[var_name]
    parent_outcomes = [outcomeSpace[var] for var in (parent_names)]
    
    # cartesian product to generate a table of all possible outcomes
    all_parent_combinations = product(*parent_outcomes)

    prob_table = odict()
    
    for i, parent_combination in enumerate(all_parent_combinations):
        parent_vars = dict(zip(parent_names, parent_combination))
        parent_index = allEqualThisIndex(data[1:], **parent_vars)
        for var_outcome in var_outcomes:
            var_index = (np.asarray(data[:-1][var_name])==var_outcome)
            if parent_index.sum()==0.0:
                prob_table[tuple(list(parent_combination)+[var_outcome])] = -math.inf
            else:
                x=(var_index & parent_index).sum()/parent_index.sum()
                prob_table[tuple(list(parent_combination)+[var_outcome])] = math.log(x) if x!=0 else -math.inf 
    return {'dom': tuple(list(parent_names)+[var_name]), 'table': prob_table}

def t_sensor_log_learn_bayes_net(G, data, outcomeSpace):
    prob_tables={}
    for node in G:
        cond=G[node]
        prob_tables[node]=t_sensor_log_estProbTable(data, node, cond, outcomeSpace)
    return prob_tables

In [5]:
# structure of rooms
graphX ={
    "c1": ["c1","c2","r7","r25"],
    "c2": ["c1","c2","c4","r34"],
    "c3": [],
    "c4": ["c2","c4","o1","r28","r29","r35"],
    "o1": [],
    "r1": [],
    "r2": ["r1","r2","r4"],
    "r3": ["r1","r3","r7"],
    "r4": ["r2","r4","r8"],
    "r5": [],
    "r6": ["c3","r5","r6"],
    "r7": ["c1","r3","r7"],
    "r8": ["r4","r8","r9"],
    "r9": ["r5","r8","r9","r13"],
    "r10": ["c3","r10"],
    "r11": ["c3","r11"],
    "r12": ["r12","r22"],
    "r13": ["r9","r13","r24"],
    "r14": ["r14","r24"],
    "r15": ["c3","r15"],
    "r16": [],
    "r17": ["c3","r17"],
    "r18": ["c3","r18"],
    "r19": ["c3","r19"],
    "r20": ["c3","r20"],
    "r21": ["c3","r21"],
    "r22": ["r12","r22","r25"],
    "r23": ["r23","r24"],
    "r24": [],
    "r25": [],
    "r26": ["r25","r26","r27"],
    "r27": ["r26","r27","r32"],
    "r28": ["c4","r28"],
    "r29": ["c4","r29","r30"],
    "r30": ["r29","r30"],
    "r31": [],
    "r32": ["r27","r31","r32","r33"],
    "r33": ["r32","r33"],
    "r34": ["c2","r34"],
    "r35": ["c4","r35"],
    "outside": ["r12","outside"]
}

# room with sensor presented
graphE={
    "r16": ["reliable_sensor1"],
    "r5": ["reliable_sensor2"],
    "r25": ["reliable_sensor3"],
    "r31": ["reliable_sensor4"],
    "o1": ["unreliable_sensor1"],
    "c3": ["unreliable_sensor2"],
    "r1": ["unreliable_sensor3"],
    "r24": ["unreliable_sensor4"],
    "r8": ["door_sensor1"],
    "r9": ["door_sensor1"],
    "c1": ["door_sensor2"],
    "c2": ["door_sensor2"],
    "r26": ["door_sensor3"],
    "r27": ["door_sensor3"],
    "c4": ["door_sensor4"],
    "r35": ["door_sensor4"]
}

graph_sensor=transposeGraph(graphE)


# list of sensors
sensors=["reliable_sensor1", "reliable_sensor2", "reliable_sensor3", "reliable_sensor4", 
        "unreliable_sensor1", "unreliable_sensor2", "unreliable_sensor3", "unreliable_sensor4", 
        "door_sensor1", "door_sensor2", "door_sensor3", "door_sensor4"]

# load data
data=pd.read_csv("data.csv", index_col=0)
col_names=list(data.columns)

# test data
first=col_names.index("r1")
truth_col=col_names[first:]

# test data into binary data
for col in truth_col:
    data[col]=data[col].apply(lambda x: 'Yes' if x>=1 else 'No')

# sensor into motion=1 and no_motion=0
sensor_col=["reliable_sensor1", "reliable_sensor2", "reliable_sensor3", "reliable_sensor4", 
        "unreliable_sensor1", "unreliable_sensor2", "unreliable_sensor3", "unreliable_sensor4"]
for col in sensor_col:
    data[col]=data[col].apply(lambda x: "Yes" if x=="motion" else "No")

door_col = ['door_sensor1','door_sensor2','door_sensor3','door_sensor4']
# cols=["r1"]
for col in door_col:
    data[col]=data[col].apply(lambda x: "No" if int(x)==0 else "Yes")

sensor_prob={}
for key in sensors:
#     print(pd.value_counts(data[key]))
    sensor_prob[key] = pd.value_counts(data[key])['Yes']/2401
data

,reliable_sensor1,reliable_sensor2,reliable_sensor3,reliable_sensor4,unreliable_sensor1,unreliable_sensor2,unreliable_sensor3,unreliable_sensor4,robot1,robot2,...,r32,r33,r34,r35,c1,c2,c3,c4,o1,outside
0,No,No,No,No,No,Yes,No,No,"('r1', 0)","('r19', 0)",...,No,No,No,No,No,No,No,No,No,Yes
1,No,No,Yes,No,No,No,No,No,"('r1', 0)","('r19', 0)",...,No,No,No,No,Yes,No,No,No,No,Yes
2,No,No,Yes,No,No,No,No,No,"('r1', 0)","('r19', 0)",...,No,No,No,No,Yes,No,No,No,No,No
3,No,No,Yes,No,No,Yes,No,No,"('r1', 0)","('c3', 0)",...,No,No,No,No,Yes,No,No,No,No,No
4,No,No,Yes,No,No,No,No,No,"('r1', 0)","('r20', 0)",...,No,No,No,No,Yes,Yes,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2396,No,No,No,No,Yes,Yes,No,Yes,"('c4', 0)","('o1', 0)",...,No,No,No,No,No,No,No,No,No,Yes
2397,No,No,No,No,No,No,No,No,"('c4', 0)","('c4', 0)",...,No,No,No,No,No,No,No,No,No,Yes
2398,No,No,No,Yes,Yes,No,No,No,"('r35', 0)","('c4', 0)",...,No,No,No,No,No,No,No,No,No,Yes
2399,No,No,No,No,No,No,No,No,"('c4', 0)","('c4', 0)",...,No,No,No,No,No,No,No,No,No,Yes


In [6]:
temp=[t+'_t' for t in truth_col]
outcomeSpaceAll=dict.fromkeys(sensors+truth_col+temp, ('Yes', 'No'))
sensor_prob=t_sensor_log_learn_bayes_net(graph_sensor, data, outcomeSpaceAll)

In [7]:
outcomeSpace=dict.fromkeys(truth_col, ('Yes', 'No'))
transition=t_log_learn_bayes_net(graphX, data[4:2324], outcomeSpaceAll)
transition

{'c1': {'c1': {'dom': ('c1', 'c1_t'),
   'table': OrderedDict([(('Yes', 'Yes'), -0.04268405908463589),
                (('Yes', 'No'), -3.175195868985665),
                (('No', 'Yes'), -2.045073898175628),
                (('No', 'No'), -0.13853891435514498)])},
  'c2': {'dom': ('c2', 'c1_t'),
   'table': OrderedDict([(('Yes', 'Yes'), -0.23141116699047823),
                (('Yes', 'No'), -1.5770344883794412),
                (('No', 'Yes'), -0.35682028221382284),
                (('No', 'No'), -1.2036337638073467)])},
  'r7': {'dom': ('r7', 'c1_t'),
   'table': OrderedDict([(('Yes', 'Yes'), -0.24923614557926788),
                (('Yes', 'No'), -1.5113855900829987),
                (('No', 'Yes'), -0.42830460007798715),
                (('No', 'No'), -1.0544410703549723)])},
  'r25': {'dom': ('r25', 'c1_t'),
   'table': OrderedDict([(('Yes', 'Yes'), -0.29316153821640645),
                (('Yes', 'No'), -1.3700338402481103),
                (('No', 'Yes'), -0.27659473181153765),
  

In [8]:
emission={}
for cond in graphE:
    child=graphE[cond]
    child=child[0]
    emission[cond]=log_estProbTable(data[4:2324], child, [cond], outcomeSpaceAll)
emission

{'r16': {'dom': ('r16', 'reliable_sensor1'),
  'table': OrderedDict([(('Yes', 'Yes'), -0.016081599218342162),
               (('Yes', 'No'), -4.13810959018886),
               (('No', 'Yes'), -3.661791734163188),
               (('No', 'No'), -0.026022105363200092)])},
 'r5': {'dom': ('r5', 'reliable_sensor2'),
  'table': OrderedDict([(('Yes', 'Yes'), -0.043216601499782265),
               (('Yes', 'No'), -3.163061045761804),
               (('No', 'Yes'), -3.277899165317662),
               (('No', 'No'), -0.0384367068410726)])},
 'r25': {'dom': ('r25', 'reliable_sensor3'),
  'table': OrderedDict([(('Yes', 'Yes'), -0.04423913233401315),
               (('Yes', 'No'), -3.140183556292536),
               (('No', 'Yes'), -3.3775875160230218),
               (('No', 'No'), -0.03472571137382963)])},
 'r31': {'dom': ('r31', 'reliable_sensor4'),
  'table': OrderedDict([(('Yes', 'Yes'), -0.01956676631828491),
               (('Yes', 'No'), -3.9436901786805185),
               (('No', 'Yes'), 

In [9]:
rev_emission={}
for node in graphE:
    cond=graphE[node]
    rev_emission[node]=log_estProbTable(data[4:2324], node, cond, outcomeSpaceAll)
rev_emission

{'r16': {'dom': ('reliable_sensor1', 'r16'),
  'table': OrderedDict([(('Yes', 'Yes'), -0.024442851943025583),
               (('Yes', 'No'), -3.7236139920935094),
               (('No', 'Yes'), -4.075751729145485),
               (('No', 'No'), -0.01712524952546337)])},
 'r5': {'dom': ('reliable_sensor2', 'r5'),
  'table': OrderedDict([(('Yes', 'Yes'), -0.21236006591318193),
               (('Yes', 'No'), -1.653773726254152),
               (('No', 'Yes'), -4.925181145555232),
               (('No', 'No'), -0.0072879031575913255)])},
 'r25': {'dom': ('reliable_sensor3', 'r25'),
  'table': OrderedDict([(('Yes', 'Yes'), -0.05932937805760315),
               (('Yes', 'No'), -2.854168709232204),
               (('No', 'Yes'), -3.669779282740517),
               (('No', 'No'), -0.02581238530740341)])},
 'r31': {'dom': ('reliable_sensor4', 'r31'),
  'table': OrderedDict([(('Yes', 'Yes'), -0.03544674899574757),
               (('Yes', 'No'), -3.3573947590037623),
               (('No', 'Yes')

In [10]:
start=dict.fromkeys(truth_col, 'No')
start['outside']='Yes'
start['r12']='Yes'
start['r22']='Yes'
print(start)

def df_to_dic(e):
    ind=list(e.index)
    e_dic={}
    for i in range(len(e)):
        e_dic[ind[i]]=e[i]
    return e_dic

{'r1': 'No', 'r2': 'No', 'r3': 'No', 'r4': 'No', 'r5': 'No', 'r6': 'No', 'r7': 'No', 'r8': 'No', 'r9': 'No', 'r10': 'No', 'r11': 'No', 'r12': 'Yes', 'r13': 'No', 'r14': 'No', 'r15': 'No', 'r16': 'No', 'r17': 'No', 'r18': 'No', 'r19': 'No', 'r20': 'No', 'r21': 'No', 'r22': 'Yes', 'r23': 'No', 'r24': 'No', 'r25': 'No', 'r26': 'No', 'r27': 'No', 'r28': 'No', 'r29': 'No', 'r30': 'No', 'r31': 'No', 'r32': 'No', 'r33': 'No', 'r34': 'No', 'r35': 'No', 'c1': 'No', 'c2': 'No', 'c3': 'No', 'c4': 'No', 'o1': 'No', 'outside': 'Yes'}


In [11]:
norm_const=(3*1+0.1*2+0.1*(len(start.keys())-5))
pred=start
e=data.loc[0][sensors]
e=df_to_dic(e)
print(e)
B={}
for room in graphX:
    if room in graphE:
        B[room]=rev_emission[room]['table'][(e[graphE[room][0]], pred[room])]+math.log(0.1/norm_const)
    else:
        if room in ['outside', 'r12', 'r22']:
            B[room]=math.log(1/norm_const)
        else:
            B[room]=math.log(0.1/norm_const)
B

{'reliable_sensor1': 'No', 'reliable_sensor2': 'No', 'reliable_sensor3': 'No', 'reliable_sensor4': 'No', 'unreliable_sensor1': 'No', 'unreliable_sensor2': 'Yes', 'unreliable_sensor3': 'No', 'unreliable_sensor4': 'No', 'door_sensor1': 'No', 'door_sensor2': 'No', 'door_sensor3': 'No', 'door_sensor4': 'No'}


{'c1': -5.549607507952356,
 'c2': -5.026831714815707,
 'c3': -6.024618507579026,
 'c4': -7.645741164999587,
 'o1': -4.290169061937502,
 'r1': -4.296648740012942,
 'r2': -4.219507705176107,
 'r3': -4.219507705176107,
 'r4': -4.219507705176107,
 'r5': -4.226795608333698,
 'r6': -4.219507705176107,
 'r7': -4.219507705176107,
 'r8': -4.262227178919038,
 'r9': -4.5414717261889015,
 'r10': -4.219507705176107,
 'r11': -4.219507705176107,
 'r12': -1.9169226121820613,
 'r13': -4.219507705176107,
 'r14': -4.219507705176107,
 'r15': -4.219507705176107,
 'r16': -4.23663295470157,
 'r17': -4.219507705176107,
 'r18': -4.219507705176107,
 'r19': -4.219507705176107,
 'r20': -4.219507705176107,
 'r21': -4.219507705176107,
 'r22': -1.9169226121820613,
 'r23': -4.219507705176107,
 'r24': -4.22374949308447,
 'r25': -4.24532009048351,
 'r26': -4.340264550641596,
 'r27': -4.5614769780856355,
 'r28': -4.219507705176107,
 'r29': -4.219507705176107,
 'r30': -4.219507705176107,
 'r31': -4.240370139830265,
 'r32

In [12]:
graph_min_E = {
    "r2":["unreliable_sensor3"],
    "r3":["unreliable_sensor3"],
    "r6":["reliable_sensor2"],
    'r10':['unreliable_sensor2'],
    'r15':['unreliable_sensor2'],
    'r11':['unreliable_sensor2'],
    'r12':['reliable_sensor3'],
    'r17':['unreliable_sensor2'],
    'r18':['unreliable_sensor2'],
    'r19':['unreliable_sensor2'],
    'r20':['unreliable_sensor2'],
    'r21':['unreliable_sensor2'],
    'r22':['reliable_sensor3'],
    'r13':['unreliable_sensor4'],
    'r14':['unreliable_sensor4'],
    'r23':['unreliable_sensor4'],
    'r32':['reliable_sensor4'],
    'r4':["unreliable_sensor3"],
    'r7':["unreliable_sensor3"],
    'r28':["unreliable_sensor1"],
    'r29':["unreliable_sensor1"],
    'r33':["reliable_sensor4"],
    'r30':['unreliable_sensor1'],
    'r34':['unreliable_sensor1'],
    'outside':['reliable_sensor3']
}

In [13]:
def calculateB(pred, prob, new_e):
    for room in prob:
        if room in graphE:
            B[room]=emission[room]['table'][(pred[room], new_e[graphE[room][0]])]+prob[room]['table'][(pred[room],)]
        else:
            x=emission[room]['table'][(pred[room], new_e[graph_min_E[room][0]])]
            p=math.log(x) if x!=0 else -math.inf
            B[room]= p + prob[room]['table'][(pred[room],)]
    return B

In [14]:
room_level1 = {"r2":{"unreliable_sensor3":["r1","r2"]},
               "r3":{"unreliable_sensor3":["r1","r3"]},
               "r6":{"reliable_sensor2":["r5","r6"]},
               'r10':{'unreliable_sensor2':['c3','r10']},
               'r15':{'unreliable_sensor2':['c3','r15']},
               'r11':{'unreliable_sensor2':['c3','r11']},
               'r17':{'unreliable_sensor2':['c3','r17']},
               'r18':{'unreliable_sensor2':['c3','r18']},
               'r19':{'unreliable_sensor2':['c3','r19']},
               'r20':{'unreliable_sensor2':['c3','r20']},
               'r21':{'unreliable_sensor2':['c3','r21']},
               'r22':{'reliable_sensor3':['r22','r25']},
               'r13':{'unreliable_sensor4':['r13','r24']},
               'r14':{'unreliable_sensor4':['r14','r24']},
               'r23':{'unreliable_sensor4':['r23','r24']},
               'r32':{'reliable_sensor4':['r31','r32']}}

room_level2 = {'r4':{"unreliable_sensor3":['r1','r2','r4']},
               'r7':{"unreliable_sensor3":['r1','r3','r7']},
               'r12':{'reliable_sensor3':['r22','r25','r12']},
               'r28':{"unreliable_sensor1":['c4','o1','r28']},
               'r29':{"unreliable_sensor1":['c4','o1','r29']},
               'r33':{"reliable_sensor4":['r31','r32','r33']},
              }
room_level2_no = {'r4':['r1','r2'],
               'r7':['r1','r3'],
               'r28':['c4','o1'],
               'r29':['c4','o1'],
               'r33':['r31','r32'],
               'r12':['r22','r25']
              }
room_level3_no = {
    'r30':['c4','o1','r29'],
    'r34':['c2','c4','o1'],
    'outside':['r12','r22','r25'],
}

room_level3 = {
    'r30':{'unreliable_sensor1':['c4','o1','r29','r30']},
    'r34':{'unreliable_sensor1':['c2','c4','o1','r34']}, 
    'outside':{'reliable_sensor3':['r12','r22','r25','outside']}
}




In [15]:
sensor_given_comb = {}
for node in room_level1:
    cond=room_level1[node][list(room_level1[node].keys())[0]]
    sen = list(room_level1[node].keys())[0]
    sensor_given_comb[node]=estProbTable(data[4:2324], sen, cond, outcomeSpaceAll)
sensor_given_comb

{'r2': {'dom': ('r1', 'r2', 'unreliable_sensor3'),
  'table': OrderedDict([(('Yes', 'Yes', 'Yes'), 0.9506172839506173),
               (('Yes', 'Yes', 'No'), 0.04938271604938271),
               (('Yes', 'No', 'Yes'), 0.9516263552960801),
               (('Yes', 'No', 'No'), 0.04837364470391994),
               (('No', 'Yes', 'Yes'), 0.1411042944785276),
               (('No', 'Yes', 'No'), 0.8588957055214724),
               (('No', 'No', 'Yes'), 0.14195979899497488),
               (('No', 'No', 'No'), 0.8580402010050251)])},
 'r3': {'dom': ('r1', 'r3', 'unreliable_sensor3'),
  'table': OrderedDict([(('Yes', 'Yes', 'Yes'), 0.9407665505226481),
               (('Yes', 'Yes', 'No'), 0.059233449477351915),
               (('Yes', 'No', 'Yes'), 0.9593392630241423),
               (('Yes', 'No', 'No'), 0.04066073697585769),
               (('No', 'Yes', 'Yes'), 0.14960629921259844),
               (('No', 'Yes', 'No'), 0.8503937007874016),
               (('No', 'No', 'Yes'), 0.1330376940

In [16]:
given1 = {}
for node in room_level1.keys():
    temp=[]
    temp1 = []
    cond=room_level1[node][list(room_level1[node].keys())[0]]
    sen = list(room_level1[node].keys())[0]
    temp.append(node)
    temp1.append(node)
    temp =list(set(cond)-set(temp))
    given1[node] = estProbTable(data[4:2324], temp[0], temp1, outcomeSpaceAll)
given1

{'r2': {'dom': ('r2', 'r1'),
  'table': OrderedDict([(('Yes', 'Yes'), 0.49846153846153846),
               (('Yes', 'No'), 0.5015384615384615),
               (('No', 'Yes'), 0.6010025062656642),
               (('No', 'No'), 0.39899749373433585)])},
 'r3': {'dom': ('r3', 'r1'),
  'table': OrderedDict([(('Yes', 'Yes'), 0.5304990757855823),
               (('Yes', 'No'), 0.46950092421441775),
               (('No', 'Yes'), 0.635702746365105),
               (('No', 'No'), 0.364297253634895)])},
 'r6': {'dom': ('r6', 'r5'),
  'table': OrderedDict([(('Yes', 'Yes'), 0.096),
               (('Yes', 'No'), 0.904),
               (('No', 'Yes'), 0.15167095115681234),
               (('No', 'No'), 0.8483290488431876)])},
 'r10': {'dom': ('r10', 'c3'),
  'table': OrderedDict([(('Yes', 'Yes'), 0.41822784810126584),
               (('Yes', 'No'), 0.5817721518987342),
               (('No', 'Yes'), 0.5942028985507246),
               (('No', 'No'), 0.4057971014492754)])},
 'r15': {'dom': ('r15', '

In [17]:
table1 = {}
for node in room_level1.keys():
    table1[node] = join(sensor_given_comb[node],given1[node],outcomeSpaceAll)
table1

{'r2': {'dom': ('r1', 'r2', 'unreliable_sensor3'),
  'table': OrderedDict([(('Yes', 'Yes', 'Yes'), 0.47384615384615386),
               (('Yes', 'Yes', 'No'), 0.024615384615384615),
               (('Yes', 'No', 'Yes'), 0.5719298245614035),
               (('Yes', 'No', 'No'), 0.029072681704260656),
               (('No', 'Yes', 'Yes'), 0.07076923076923075),
               (('No', 'Yes', 'No'), 0.43076923076923074),
               (('No', 'No', 'Yes'), 0.05664160401002507),
               (('No', 'No', 'No'), 0.34235588972431075)])},
 'r3': {'dom': ('r1', 'r3', 'unreliable_sensor3'),
  'table': OrderedDict([(('Yes', 'Yes', 'Yes'), 0.4990757855822552),
               (('Yes', 'Yes', 'No'), 0.031423290203327174),
               (('Yes', 'No', 'Yes'), 0.6098546042003231),
               (('Yes', 'No', 'No'), 0.025848142164781908),
               (('No', 'Yes', 'Yes'), 0.07024029574861368),
               (('No', 'Yes', 'No'), 0.39926062846580407),
               (('No', 'No', 'Yes'), 0.04

In [18]:
sum1 = {}
for node in room_level1.keys():
    temp=[]
    cond=room_level1[node][list(room_level1[node].keys())[0]]
    sen = list(room_level1[node].keys())[0]
    temp.append(node)
    temp =list(set(cond)-set(temp))
    sum1[node] = marginalize(table1[node],temp[0],outcomeSpaceAll)
sum1 

{'r2': {'dom': ('r2', 'unreliable_sensor3'),
  'table': OrderedDict([(('Yes', 'Yes'), 0.5446153846153846),
               (('Yes', 'No'), 0.45538461538461533),
               (('No', 'Yes'), 0.6285714285714286),
               (('No', 'No'), 0.3714285714285714)])},
 'r3': {'dom': ('r3', 'unreliable_sensor3'),
  'table': OrderedDict([(('Yes', 'Yes'), 0.5693160813308689),
               (('Yes', 'No'), 0.43068391866913125),
               (('No', 'Yes'), 0.6583198707592892),
               (('No', 'No'), 0.34168012924071084)])},
 'r6': {'dom': ('r6', 'reliable_sensor2'),
  'table': OrderedDict([(('Yes', 'Yes'), 0.13866666666666666),
               (('Yes', 'No'), 0.8613333333333334),
               (('No', 'Yes'), 0.17480719794344474),
               (('No', 'No'), 0.8251928020565552)])},
 'r10': {'dom': ('r10', 'unreliable_sensor2'),
  'table': OrderedDict([(('Yes', 'Yes'), 0.4627848101265823),
               (('Yes', 'No'), 0.5372151898734177),
               (('No', 'Yes'), 0.62898550

In [19]:
sensor_given_comb2 = {}
for node in room_level2:
    cond=room_level2[node][list(room_level2[node].keys())[0]]
    sen = list(room_level2[node].keys())[0]
    sensor_given_comb2[node]=estProbTable(data[4:2324], sen, cond, outcomeSpaceAll)
sensor_given_comb2

{'r4': {'dom': ('r1', 'r2', 'r4', 'unreliable_sensor3'),
  'table': OrderedDict([(('Yes', 'Yes', 'Yes', 'Yes'), 0.9285714285714286),
               (('Yes', 'Yes', 'Yes', 'No'), 0.07142857142857142),
               (('Yes', 'Yes', 'No', 'Yes'), 0.9583333333333334),
               (('Yes', 'Yes', 'No', 'No'), 0.041666666666666664),
               (('Yes', 'No', 'Yes', 'Yes'), 0.9535398230088495),
               (('Yes', 'No', 'Yes', 'No'), 0.046460176991150445),
               (('Yes', 'No', 'No', 'Yes'), 0.9504685408299867),
               (('Yes', 'No', 'No', 'No'), 0.049531459170013385),
               (('No', 'Yes', 'Yes', 'Yes'), 0.18518518518518517),
               (('No', 'Yes', 'Yes', 'No'), 0.8148148148148148),
               (('No', 'Yes', 'No', 'Yes'), 0.1323529411764706),
               (('No', 'Yes', 'No', 'No'), 0.8676470588235294),
               (('No', 'No', 'Yes', 'Yes'), 0.10465116279069768),
               (('No', 'No', 'Yes', 'No'), 0.8953488372093024),
            

In [20]:
given_1_2 = {}
for node in room_level2.keys():
    cond=room_level2[node][list(room_level2[node].keys())[0]]
    given_1_2[node] = estProbTable(data[4:2324], cond[0], cond[1:3], outcomeSpaceAll)
given_1_2

{'r4': {'dom': ('r2', 'r4', 'r1'),
  'table': OrderedDict([(('Yes', 'Yes', 'Yes'), 0.6086956521739131),
               (('Yes', 'Yes', 'No'), 0.391304347826087),
               (('Yes', 'No', 'Yes'), 0.46875),
               (('Yes', 'No', 'No'), 0.53125),
               (('No', 'Yes', 'Yes'), 0.7243589743589743),
               (('No', 'Yes', 'No'), 0.27564102564102566),
               (('No', 'No', 'Yes'), 0.5448577680525164),
               (('No', 'No', 'No'), 0.4551422319474836)])},
 'r7': {'dom': ('r3', 'r7', 'r1'),
  'table': OrderedDict([(('Yes', 'Yes', 'Yes'), 0.5891472868217055),
               (('Yes', 'Yes', 'No'), 0.4108527131782946),
               (('Yes', 'No', 'Yes'), 0.38311688311688313),
               (('Yes', 'No', 'No'), 0.6168831168831169),
               (('No', 'Yes', 'Yes'), 0.6296296296296297),
               (('No', 'Yes', 'No'), 0.37037037037037035),
               (('No', 'No', 'Yes'), 0.6772151898734177),
               (('No', 'No', 'No'), 0.322784810126

In [21]:
given_1_1 = {}
for node in room_level2.keys():
    cond=room_level2[node][list(room_level2[node].keys())[0]]
    given_1_1[node] = estProbTable(data[4:2324], cond[1], cond[2:3], outcomeSpaceAll)
given_1_1 

{'r4': {'dom': ('r4', 'r2'),
  'table': OrderedDict([(('Yes', 'Yes'), 0.09956709956709957),
               (('Yes', 'No'), 0.9004329004329005),
               (('No', 'Yes'), 0.15734480639213275),
               (('No', 'No'), 0.8426551936078672)])},
 'r7': {'dom': ('r7', 'r3'),
  'table': OrderedDict([(('Yes', 'Yes'), 0.4174757281553398),
               (('Yes', 'No'), 0.5825242718446602),
               (('No', 'Yes'), 0.6609442060085837),
               (('No', 'No'), 0.33905579399141633)])},
 'r12': {'dom': ('r12', 'r25'),
  'table': OrderedDict([(('Yes', 'Yes'), 0.45454545454545453),
               (('Yes', 'No'), 0.5454545454545454),
               (('No', 'Yes'), 0.3614738805970149),
               (('No', 'No'), 0.6385261194029851)])},
 'r28': {'dom': ('r28', 'o1'),
  'table': OrderedDict([(('Yes', 'Yes'), 0.7291666666666666),
               (('Yes', 'No'), 0.2708333333333333),
               (('No', 'Yes'), 0.46963028169014087),
               (('No', 'No'), 0.5303697183098591

In [22]:
table2 = {}
temp2={}
for node in room_level2.keys():
    temp2[node] = join(sensor_given_comb2[node],given_1_2[node],outcomeSpaceAll)
    table2[node] = join(temp2[node],given_1_1[node],outcomeSpaceAll)
table2

{'r4': {'dom': ('r1', 'r2', 'r4', 'unreliable_sensor3'),
  'table': OrderedDict([(('Yes', 'Yes', 'Yes', 'Yes'), 0.05627705627705629),
               (('Yes', 'Yes', 'Yes', 'No'), 0.004329004329004329),
               (('Yes', 'Yes', 'No', 'Yes'), 0.07068223724646588),
               (('Yes', 'Yes', 'No', 'No'), 0.003073140749846343),
               (('Yes', 'No', 'Yes', 'Yes'), 0.621933621933622),
               (('Yes', 'No', 'Yes', 'No'), 0.030303030303030304),
               (('Yes', 'No', 'No', 'Yes'), 0.43638598647818067),
               (('Yes', 'No', 'No', 'No'), 0.022741241548862937),
               (('No', 'Yes', 'Yes', 'Yes'), 0.007215007215007215),
               (('No', 'Yes', 'Yes', 'No'), 0.031746031746031744),
               (('No', 'Yes', 'No', 'Yes'), 0.011063306699446834),
               (('No', 'Yes', 'No', 'No'), 0.0725261216963737),
               (('No', 'No', 'Yes', 'Yes'), 0.02597402597402598),
               (('No', 'No', 'Yes', 'No'), 0.22222222222222227),
   

In [23]:
sum2 = {}
temp_sum2 = {}
for node in room_level2.keys():
    temp=[]
    cond=room_level2[node][list(room_level2[node].keys())[0]]
    temp_sum2[node] = marginalize(table2[node],cond[0],outcomeSpaceAll)
    sum2[node] =  marginalize(temp_sum2[node],cond[1],outcomeSpaceAll)
sum2 

{'r4': {'dom': ('r4', 'unreliable_sensor3'),
  'table': OrderedDict([(('Yes', 'Yes'), 0.7113997113997115),
               (('Yes', 'No'), 0.2886002886002887),
               (('No', 'Yes'), 0.5765212046711738),
               (('No', 'No'), 0.42347879532882604)])},
 'r7': {'dom': ('r7', 'unreliable_sensor3'),
  'table': OrderedDict([(('Yes', 'Yes'), 0.6353829557713053),
               (('Yes', 'No'), 0.3646170442286947),
               (('No', 'Yes'), 0.5429184549356223),
               (('No', 'No'), 0.4570815450643777)])},
 'r12': {'dom': ('r12', 'reliable_sensor3'),
  'table': OrderedDict([(('Yes', 'Yes'), 0.43749999999999994),
               (('Yes', 'No'), 0.5625),
               (('No', 'Yes'), 0.36893656716417905),
               (('No', 'No'), 0.6310634328358209)])},
 'r28': {'dom': ('r28', 'unreliable_sensor1'),
  'table': OrderedDict([(('Yes', 'Yes'), 0.7291666666666666),
               (('Yes', 'No'), 0.27083333333333337),
               (('No', 'Yes'), 0.5154049295774648),


In [24]:
sensor_given_comb3= {}
for node in room_level3:
    cond=room_level3[node][list(room_level3[node].keys())[0]]
    sen = list(room_level3[node].keys())[0]
    sensor_given_comb3[node]=estProbTable(data, sen, cond, outcomeSpaceAll)
sensor_given_comb3

{'r30': {'dom': ('c4', 'o1', 'r29', 'r30', 'unreliable_sensor1'),
  'table': OrderedDict([(('Yes', 'Yes', 'Yes', 'Yes', 'Yes'), 0),
               (('Yes', 'Yes', 'Yes', 'Yes', 'No'), 0),
               (('Yes', 'Yes', 'Yes', 'No', 'Yes'), 1.0),
               (('Yes', 'Yes', 'Yes', 'No', 'No'), 0.0),
               (('Yes', 'Yes', 'No', 'Yes', 'Yes'), 0.0),
               (('Yes', 'Yes', 'No', 'Yes', 'No'), 1.0),
               (('Yes', 'Yes', 'No', 'No', 'Yes'), 0.9301227573182247),
               (('Yes', 'Yes', 'No', 'No', 'No'), 0.06987724268177525),
               (('Yes', 'No', 'Yes', 'Yes', 'Yes'), 0.15384615384615385),
               (('Yes', 'No', 'Yes', 'Yes', 'No'), 0.8461538461538461),
               (('Yes', 'No', 'Yes', 'No', 'Yes'), 0.23255813953488372),
               (('Yes', 'No', 'Yes', 'No', 'No'), 0.7674418604651163),
               (('Yes', 'No', 'No', 'Yes', 'Yes'), 0.6666666666666666),
               (('Yes', 'No', 'No', 'Yes', 'No'), 0.3333333333333333),
     

In [25]:
given3_1_3={}
for node in room_level3.keys():
    cond=room_level3[node][list(room_level3[node].keys())[0]]
    given3_1_3[node] = estProbTable(data[4:2324], cond[0], cond[1:4], outcomeSpaceAll)
given3_1_2={}
for node in room_level3.keys():
    cond=room_level3[node][list(room_level3[node].keys())[0]]
    given3_1_2[node] = estProbTable(data, cond[1], cond[2:4], outcomeSpaceAll)

given3_1_1={}
for node in room_level3.keys():
    cond=room_level3[node][list(room_level3[node].keys())[0]]
    given3_1_1[node] = estProbTable(data, cond[2], cond[3:4], outcomeSpaceAll)
print(given3_1_1)
print(given3_1_2)
print(given3_1_3)

{'r30': {'dom': ('r30', 'r29'), 'table': OrderedDict([(('Yes', 'Yes'), 0.65), (('Yes', 'No'), 0.35), (('No', 'Yes'), 0.025619487610247797), (('No', 'No'), 0.9743805123897522)])}, 'r34': {'dom': ('r34', 'o1'), 'table': OrderedDict([(('Yes', 'Yes'), 0.48530646515533166), (('Yes', 'No'), 0.5146935348446684), (('No', 'Yes'), 0.43305785123966944), (('No', 'No'), 0.5669421487603306)])}, 'outside': {'dom': ('outside', 'r25'), 'table': OrderedDict([(('Yes', 'Yes'), 0.08264462809917356), (('Yes', 'No'), 0.9173553719008265), (('No', 'Yes'), 0.3714912280701754), (('No', 'No'), 0.6285087719298246)])}}
{'r30': {'dom': ('r29', 'r30', 'o1'), 'table': OrderedDict([(('Yes', 'Yes', 'Yes'), 0.0), (('Yes', 'Yes', 'No'), 1.0), (('Yes', 'No', 'Yes'), 0.26229508196721313), (('Yes', 'No', 'No'), 0.7377049180327869), (('No', 'Yes', 'Yes'), 0.14285714285714285), (('No', 'Yes', 'No'), 0.8571428571428571), (('No', 'No', 'Yes'), 0.4676724137931034), (('No', 'No', 'No'), 0.5323275862068966)])}, 'r34': {'dom': ('o1'

In [26]:
table3 = {}
sum3={}
for node in room_level3.keys():
    temp3={}
    temp4 ={}
    temp_sum3 = {}
    temp_sum4 = {}
    cond=room_level3[node][list(room_level3[node].keys())[0]]
    temp3[node] = join(sensor_given_comb3[node],given3_1_3[node],outcomeSpaceAll)
    temp4[node] = join(temp3[node],given3_1_2[node],outcomeSpaceAll)
    table3[node] = join(temp4[node],given3_1_1[node],outcomeSpaceAll)
    temp_sum3[node] = marginalize(table3[node],cond[0],outcomeSpaceAll)
    temp_sum4[node] =  marginalize(temp_sum3[node],cond[1],outcomeSpaceAll)
    sum3[node] =  marginalize(temp_sum4[node],cond[2],outcomeSpaceAll)
sum3

{'r30': {'dom': ('r30', 'unreliable_sensor1'),
  'table': OrderedDict([(('Yes', 'Yes'), 0.3),
               (('Yes', 'No'), 0.7000000000000001),
               (('No', 'Yes'), 0.5079099631178953),
               (('No', 'No'), 0.4920900368821047)])},
 'r34': {'dom': ('r34', 'unreliable_sensor1'),
  'table': OrderedDict([(('Yes', 'Yes'), 0.5289672544080604),
               (('Yes', 'No'), 0.47103274559193964),
               (('No', 'Yes'), 0.48216242658915553),
               (('No', 'No'), 0.5178375734108446)])},
 'outside': {'dom': ('outside', 'reliable_sensor3'),
  'table': OrderedDict([(('Yes', 'Yes'), 0.09386068476977569),
               (('Yes', 'No'), 0.9061393152302243),
               (('No', 'Yes'), 0.3771588820255006),
               (('No', 'No'), 0.6228411179744994)])}}

In [27]:
emission.update(sum1)
emission.update(sum2)
emission.update(sum3)
emission['r2']

{'dom': ('r2', 'unreliable_sensor3'),
 'table': OrderedDict([(('Yes', 'Yes'), 0.5446153846153846),
              (('Yes', 'No'), 0.45538461538461533),
              (('No', 'Yes'), 0.6285714285714286),
              (('No', 'No'), 0.3714285714285714)])}

In [28]:
with open('emission.pickle', 'wb') as handle:
    pickle.dump(emission, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [29]:
with open('transition.pickle', 'wb') as handle:
    pickle.dump(transition, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [30]:
with open('reversed_emission.pickle', 'wb') as handle:
    pickle.dump(rev_emission, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [31]:
with open('sensor_prob.pickle', 'wb') as handle:
    pickle.dump(sensor_prob, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [32]:
sensor_prob

{'reliable_sensor1': {'dom': ('r16', 'reliable_sensor1'),
  'table': OrderedDict([(('Yes', 'Yes'), -0.028962842161623575),
               (('Yes', 'No'), -3.5561880447391387),
               (('No', 'Yes'), -3.2689074541256975),
               (('No', 'No'), -0.03879069809612828)])},
 'reliable_sensor2': {'dom': ('r5', 'reliable_sensor2'),
  'table': OrderedDict([(('Yes', 'Yes'), -0.5189146466390743),
               (('Yes', 'No'), -0.9042785754261515),
               (('No', 'Yes'), -2.356706018515026),
               (('No', 'No'), -0.09952397530145446)])},
 'reliable_sensor3': {'dom': ('r25', 'reliable_sensor3'),
  'table': OrderedDict([(('Yes', 'Yes'), -0.17141277970495358),
               (('Yes', 'No'), -1.8481631401593503),
               (('No', 'Yes'), -2.330848558266905),
               (('No', 'No'), -0.10226887858335514)])},
 'reliable_sensor4': {'dom': ('r31', 'reliable_sensor4'),
  'table': OrderedDict([(('Yes', 'Yes'), -0.030797985887785404),
               (('Yes', 'No'

In [33]:
graph_sensor

{'reliable_sensor1': ['r16'],
 'reliable_sensor2': ['r5'],
 'reliable_sensor3': ['r25'],
 'reliable_sensor4': ['r31'],
 'unreliable_sensor1': ['o1'],
 'unreliable_sensor2': ['c3'],
 'unreliable_sensor3': ['r1'],
 'unreliable_sensor4': ['r24'],
 'door_sensor1': ['r8', 'r9'],
 'door_sensor2': ['c1', 'c2'],
 'door_sensor3': ['r26', 'r27'],
 'door_sensor4': ['c4', 'r35']}

In [34]:
pred_prob={}

for i in range(1, len(data)):    
    new_e=data.loc[0][sensors]
    new_e=df_to_dic(new_e)
    if len(pred_prob.keys())!=0:
        B=calculateB(pred, pred_prob, new_e)

    for room in graphX:
        temp_prob={'dom': (room, ),
                     'table': odict([])}
        if len(graphX[room])!=0:
            for neighbour in graphX[room]:
                temp_prob['table'][('Yes', )]=temp_prob['table'].get(('Yes', ), 0)+B[room]+transition[room][neighbour]['table'][(pred[room], 'Yes')]
                temp_prob['table'][('No', )]=temp_prob['table'].get(('No', ), 0)+B[room]+transition[room][neighbour]['table'][(pred[room], 'No')]

        else:
            temp_prob['table'][('Yes', )]=rev_emission[room]['table'][(e[graphE[room][0]],'Yes')]
            temp_prob['table'][('No', )]=rev_emission[room]['table'][(e[graphE[room][0]],'No')]
        pred_prob[room]=temp_prob.copy()
        if temp_prob['table'][('Yes', )] > temp_prob['table'][('No', )]:
            pred[room]='Yes'
        else:
            pred[room]='No'